# SignXAI2 with TensorFlow - Basic Usage (VGG16)

This tutorial demonstrates how to use the SignXAI2 package with TensorFlow to explain a VGG16 image classification model. We'll walk through:

1. Setting up the environment
2. Loading a pre-trained VGG16 model and sample image
3. Generating explanations using the unified API
4. Visualizing and comparing the results

## Setup Requirements

**Important**: SignXAI2 requires Python 3.9 or 3.10 (Python 3.11+ is not supported)

Since you're running this tutorial, you should already have cloned the signxai2 repository. From the repository root directory:

### Using conda:
```bash
# Create environment with Python 3.10
conda create -n signxai2 python=3.10
conda activate signxai2

# Install SignXAI2 with TensorFlow support
pip install signxai2[tensorflow]

# Download models and example data
git lfs install
bash ./prepare.sh
```

### Using venv:
```bash
# Create virtual environment
python3.10 -m venv signxai2_env
source signxai2_env/bin/activate  # On Windows: signxai2_env\Scripts\activate

# Install SignXAI2 with TensorFlow support
pip install signxai2[tensorflow]

# Download models and example data
git lfs install
bash ./prepare.sh
```

Let's get started!

## 1. Import Libraries

In [ ]:
import os
import sys
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow.keras.applications.vgg16 import preprocess_input, decode_predictions

# SignXAI2 unified API imports
from signxai import explain, list_methods
from signxai.utils.utils import load_image

## 2. Set Up Paths

In [ ]:
# Set up data and model paths
_THIS_DIR = os.path.dirname(os.path.abspath("__file__"))
DATA_DIR = os.path.realpath(os.path.join(_THIS_DIR, "..", "..", "data"))
VGG16_MODEL_PATH = os.path.join(DATA_DIR, "models", "tensorflow", "VGG16", "model.h5")
IMAGE_PATH = os.path.join(DATA_DIR, "images", "example.jpg")

# Check that files exist
assert os.path.exists(VGG16_MODEL_PATH), f"VGG16 model not found at {VGG16_MODEL_PATH}"
assert os.path.exists(IMAGE_PATH), f"Image not found at {IMAGE_PATH}"

## 3. Load VGG16 Model and Image

In [ ]:
# Load the VGG16 model
try:
    # First try loading our pre-saved model
    vgg16_model = tf.keras.models.load_model(VGG16_MODEL_PATH)
    print("Loaded pre-saved VGG16 model")
except Exception as e:
    # If that fails, load the model from Keras applications
    print(f"Could not load saved model: {e}\nLoading from Keras applications instead")
    vgg16_model = tf.keras.applications.VGG16(weights='imagenet', include_top=True)
    
vgg16_model.summary()

In [ ]:
# Load and preprocess image specifically for VGG16
original_img, preprocessed_img = load_image(IMAGE_PATH, target_size=(224, 224), expand_dims=True)

# Display the original image
plt.figure(figsize=(6, 6))
plt.imshow(original_img)
plt.axis('off')
plt.title('Sample Image')
plt.show()

## 4. Predict the Class with VGG16

In [ ]:
# Make a prediction with VGG16
predictions = vgg16_model.predict(preprocessed_img)
predicted_class = np.argmax(predictions[0])

# Decode the prediction (VGG16 is trained on ImageNet)
try:
    decoded_predictions = decode_predictions(predictions, top=3)[0]
    print("Top 3 predictions:")
    for i, (imagenet_id, label, score) in enumerate(decoded_predictions):
        print(f"{i+1}: {label} ({score:.4f})")
    
    # Set the class label
    class_name = decoded_predictions[0][1]
except:
    class_name = f"Class {predicted_class}"
    print(f"Predicted class index: {predicted_class}")

print(f"\nExplaining prediction: {class_name} (class index: {predicted_class})")

## 5. Generate Explanations with SignXAI

Now let's use SignXAI to explain the VGG16 model's prediction using different methods.

In [ ]:
# List all available methods in SignXAI2
print("Available methods in SignXAI2:")
available_methods = list_methods()
print(f"Total methods: {len(available_methods)}")
print("\nSome common methods:")
for method in ['gradient', 'gradient_x_input', 'grad_cam', 'guided_backprop', 'lrp_z']:
    if method in available_methods:
        print(f"  - {method}")

In [ ]:
# Define the methods we want to use for VGG16 explanation
# Using the unified API method names
methods_to_test = [
    'gradient',
    'gradient_x_input',
    'grad_cam',
    'guided_backprop', 
    'lrp_z',
    'lrp_epsilon_0_1'
]

# Additional parameters for specific methods
method_params = {
    'grad_cam': {'layer_name': 'block5_conv3'}  # VGG16-specific last conv layer
}

# Storage for explanations
explanations = {}

In [ ]:
# Generate explanations using the unified API
for method_name in methods_to_test:
    print(f"Generating {method_name} explanation...")
    
    # Get method-specific parameters
    params = method_params.get(method_name, {})
    
    # Use the unified explain API
    explanation = explain(
        model=vgg16_model,
        x=preprocessed_img,
        method_name=method_name,
        target_class=predicted_class,
        **params
    )
    
    explanations[method_name] = explanation
    
print("All explanations generated!")

## 6. Visualize VGG16 Explanations

In [ ]:
# Utility function for prettier visualization
def preprocess_explanation(explanation):
    # Remove batch dimension if present
    if explanation.ndim == 4:
        explanation = explanation[0]
    
    # Convert to RGB if single channel
    if explanation.ndim == 2:
        explanation = np.expand_dims(explanation, axis=-1)
        explanation = np.repeat(explanation, 3, axis=-1)
    elif explanation.shape[-1] == 1:
        explanation = np.repeat(explanation, 3, axis=-1)
    
    # Use absolute values for visualization
    abs_explanation = np.abs(explanation)
    
    # Normalize for visualization
    if abs_explanation.max() > 0:
        abs_explanation = abs_explanation / abs_explanation.max()
    
    return abs_explanation

In [ ]:
# Create figure for visualization
fig, axes = plt.subplots(2, 3, figsize=(15, 10))
axes = axes.flatten()

for i, (method_name, explanation) in enumerate(explanations.items()):
    processed_explanation = preprocess_explanation(explanation)
    
    # Simple overlay visualization
    axes[i].imshow(original_img)
    axes[i].imshow(processed_explanation, alpha=0.5, cmap='hot')
    axes[i].set_title(method_name.replace('_', ' ').title())
    axes[i].axis('off')

plt.suptitle(f"VGG16 Explanations for class: {class_name}", fontsize=16)
plt.tight_layout()
plt.subplots_adjust(top=0.9)  # Adjust for the suptitle
plt.show()

## 7. Interpret the VGG16 Results

Let's interpret what we're seeing in these explanation methods for the VGG16 model:

- **Gradient**: Shows pixel-level importance via the gradient of the output with respect to input. In VGG16, this often highlights edges and textures.

- **Gradient × Input**: Enhances gradient by multiplication with input values. This tends to focus more on the regions where both the gradient and input values are high.

- **GradCAM**: Uses the last convolutional layer of VGG16 (block5_conv3) to produce a coarse localization map highlighting important regions for the prediction.

- **Guided Backprop**: Creates sharper feature visualizations by modifying the backpropagation signal through ReLU layers. It's particularly effective for VGG16 which has many ReLU activations.

- **LRP-Z**: Layer-wise Relevance Propagation with the Z-rule propagates the prediction backward through the network to identify relevant input features.

- **LRP-Epsilon**: A variant of LRP that adds a stabilizing term (epsilon) to avoid division by zero, producing slightly different attribution maps.

Each method highlights different aspects of how VGG16 processes the image to make its prediction.

## 8. Conclusion

In this tutorial, we've learned how to:
- Set up SignXAI with TensorFlow
- Load and prepare a pre-trained VGG16 model for explanation
- Apply various explainability methods to understand VGG16 predictions
- Visualize and interpret the results

VGG16 is an excellent model for explainability demonstrations because of its straightforward architecture. The clear convolutional structure makes it easier to interpret how different parts of the image influence the model's predictions.

For more advanced techniques and detailed explanations of other models, check out the other tutorials in this series.